In [ ]:
import shutil

# Replace 'your_folder' with the folder you want to download
folder_path = "/content/la_s"
zip_path = "/content/la_s.zip"

# Zip the folder
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_path)

# Download the zipped folder
from google.colab import files
files.download(zip_path)

In [ ]:
import shutil

# Replace 'your_folder' with the folder you want to download
folder_path = "/content/la_t"
zip_path = "/content/la_t.zip"

# Zip the folder
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_path)

# Download the zipped folder
from google.colab import files
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Load Dataset

In [ ]:
!pip install sklearn_crfsuite

In [ ]:
!git clone https://github.com/adapter-hub/adapters.git
!cd adapters

fatal: destination path 'adapters' already exists and is not an empty directory.


In [ ]:
!pip install datasets conllu

In [ ]:
pip install transformers datasets accelerate


In [ ]:
!pip install adapters

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from adapters import AutoAdapterModel
from transformers import AutoTokenizer
from adapters import AdapterTrainer
from transformers import TrainingArguments
from transformers import AutoConfig
from adapters import AdapterConfig
from transformers import TrainingArguments, Trainer
from datasets import Dataset, concatenate_datasets

In [ ]:
%mkdir Ug/

mkdir: cannot create directory ‘Ug/’: File exists


In [ ]:
import conllu

def extract_corpus(conllu_path):
    """Extracts transliterated sentences from a CONLLU file."""
    translit_sentences = []
    original_sentences = []
    with open(conllu_path, "r", encoding="utf-8") as f:
        data = conllu.parse(f.read())

    for sentence in data:
        original = sentence.metadata.get('text', '')
        translit = sentence.metadata.get('translit', '')
        if translit:
            translit_sentences.append(translit.strip())
        if original:
            original_sentences.append(original.strip())
    return original_sentences, translit_sentences

def original_corpus(files):
    corpus = {}
    for split, file_path in files.items():
        original, translit = extract_corpus(file_path)
        corpus[split] = {"original": original, "translit": translit}

    for split in ["train", "dev", "test"]:
        with open(f"ug_{split}_original.txt", "w", encoding="utf-8") as f:
            f.write("\n".join(corpus[split]["original"]) + "\n")
        with open(f"ug_{split}_transliterated.txt", "w", encoding="utf-8") as f:
            f.write("\n".join(corpus[split]["translit"]) + "\n")

    return corpus["train"]["original"], corpus["train"]["translit"]

files = {
        "train": "ug_udt-ud-train.conllu",
        "dev": "ug_udt-ud-dev.conllu",
        "test": "ug_udt-ud-test.conllu"
    }
original_texts, transliterated_texts = original_corpus(files)

In [ ]:
original_texts[4]

'نەشپۈت كۆچىتىنى يۇلىۋېتىپ ئۆرۈك كۆچىتى تىكىپتۇ.'

In [ ]:
transliterated_texts[4]

'neshpüt köchitini yuliwëtip örük köchiti tikiptu.'

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
additional_tokens = ["<new_token1>", "<new_token2>"]
tokenizer.add_tokens(additional_tokens)

def tokenize(text):
    return tokenizer(
        text,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import os
# Force CPU-only mode
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # For better error messages

import torch
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
from adapters import AutoAdapterModel, Stack, AdapterConfig
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling

# ========== Device Setup ==========
# device = torch.device("cpu")  # Force CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Using device:", device)

# ========== Keep Original Dataset Intact ==========
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {"text": self.texts[idx]}

# ========== Model Loading with CPU Enforcement ==========
teacher_model = AutoModelForMaskedLM.from_pretrained(
    "coppercitylabs/uzbert-base-uncased"
).to(device).eval()

model = AutoAdapterModel.from_pretrained(
    "distilbert-base-multilingual-cased"
).to(device)

# ========== Tokenizers ==========
teacher_tokenizer = AutoTokenizer.from_pretrained("coppercitylabs/uzbert-base-uncased")
student_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

# ========== Adapter Setup (with device enforcement) ==========
lang_config = AdapterConfig.load(
    "pfeiffer",
    reduction_factor=16,
    leave_out=[5]
)

# Add adapters and force device placement
model.add_adapter("la_s", config=lang_config)
model.add_adapter("la_t", config=lang_config)
model = model.to(device)  # Critical: Re-affirm device placement after adding adapters
model.train_adapter(["la_s", "la_t"])

# ========== Verify Adapter Device Placement ==========
# Add debug checks for adapter parameters
# print("\nDevice verification:")
# for name, param in model.named_parameters():
#     if 'la_s' in name or 'la_t' in name:
#         print(f"{name}: {param.device}")

# ========== Data Collator ==========
data_collator = DataCollatorForLanguageModeling(
    tokenizer=student_tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# ========== Training Utilities ==========


def safe_tokenize(texts, tokenizer, is_student=False):
    """Handles tokenization with proper device placement"""
    if is_student:
        # Tokenize on CPU first
        tokens = tokenizer(
            texts,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=512
        )

        # Collate on CPU
        masked = data_collator([{"input_ids": ids} for ids in tokens["input_ids"]])

        # Move all tensors to device
        return {k: v.to(device) for k, v in masked.items()}
    else:
        # Teacher processing stays on GPU
        tokens = tokenizer(
            texts,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=512
        ).to(device)
        return tokens


def distillation_loss(s_repr, t_repr):
    return F.mse_loss(s_repr, t_repr)

# ========== Training Loop ==========
def train_adapter(adapter_name, dataloader, num_epochs=10):
    model.train_adapter(adapter_name)
    model.set_active_adapters(Stack(adapter_name))

    optimizer = AdamW(model.parameters(), lr=1e-4)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in dataloader:
            texts = batch["text"]

            # Student processing
            student_inputs = safe_tokenize(texts, student_tokenizer, is_student=True)

            # Teacher processing
            with torch.no_grad():
                teacher_inputs = safe_tokenize(texts, teacher_tokenizer)
                teacher_outputs = teacher_model(**teacher_inputs, output_hidden_states=True)
                t_repr = teacher_outputs.hidden_states[-1][:, 0, :]

            # print(f"Model device: {next(model.parameters()).device}")
            # print(f"Input IDs device: {student_inputs['input_ids'].device}")
            # print(f"Attention mask device: {student_inputs['attention_mask'].device}")

            # Student forward
            student_outputs = model(
                input_ids=student_inputs["input_ids"],
                attention_mask=student_inputs["attention_mask"],
                output_hidden_states=True
            )
            s_repr = student_outputs.hidden_states[-1][:, 0, :]

            # Loss computation
            loss = distillation_loss(s_repr, t_repr)
            loss.backward()
            # optimizer.step()
            # optimizer.zero_grad()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} | Loss: {total_loss/len(dataloader):.4f}")


original_dataset = TextDataset(original_texts)
transliterated_dataset = TextDataset(transliterated_texts)

original_dataloader = DataLoader(original_dataset, batch_size=16, shuffle=True)
transliterated_dataloader = DataLoader(transliterated_dataset, batch_size=16, shuffle=True)

# ========== Train ==========
print("Training la_s adapter...")
train_adapter("la_s", original_dataloader, num_epochs=10)

print("Training la_t adapter...")
train_adapter("la_t", transliterated_dataloader, num_epochs=10)

# ========== Save ==========
model.save_adapter("./la_s", "la_s", with_head=False)
model.save_adapter("./la_t", "la_t", with_head=False)
print("Adapters saved successfully")

Using device: cuda:0


/usr/local/lib/python3.11/dist-packages/adapters/composition.py:225: FutureWarning: Passing list objects for adapter activation is deprecated. Please use Stack or Fuse explicitly.
  warnings.warn(


Training la_s adapter...
Epoch 1 | Loss: 1.1347
Epoch 2 | Loss: 1.1293
Epoch 3 | Loss: 1.1390
Epoch 4 | Loss: 1.1342
Epoch 5 | Loss: 1.1312
Epoch 6 | Loss: 1.1323
Epoch 7 | Loss: 1.1268
Epoch 8 | Loss: 1.1380
Epoch 9 | Loss: 1.1327
Epoch 10 | Loss: 1.1268
Training la_t adapter...
Epoch 1 | Loss: 1.1076
Epoch 2 | Loss: 1.1082
Epoch 3 | Loss: 1.1118
Epoch 4 | Loss: 1.1039
Epoch 5 | Loss: 1.1068
Epoch 6 | Loss: 1.1087
Epoch 7 | Loss: 1.1089
Epoch 8 | Loss: 1.1104
Epoch 9 | Loss: 1.1075
Epoch 10 | Loss: 1.1080
Adapters saved successfully


## Model Training

In [ ]:
#  adapter configurations
print("\nla_s config:", model.get_adapter("la_s"))
print("la_t config:", model.get_adapter("la_t"))


la_s config: {0: {'output_adapter': Adapter(
  (non_linearity): Activation_Function_Class(
    (f): ReLU()
  )
  (adapter_down): Sequential(
    (0): Linear(in_features=768, out_features=48, bias=True)
    (1): Activation_Function_Class(
      (f): ReLU()
    )
  )
  (adapter_up): Linear(in_features=48, out_features=768, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)}, 1: {'output_adapter': Adapter(
  (non_linearity): Activation_Function_Class(
    (f): ReLU()
  )
  (adapter_down): Sequential(
    (0): Linear(in_features=768, out_features=48, bias=True)
    (1): Activation_Function_Class(
      (f): ReLU()
    )
  )
  (adapter_up): Linear(in_features=48, out_features=768, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)}, 2: {'output_adapter': Adapter(
  (non_linearity): Activation_Function_Class(
    (f): ReLU()
  )
  (adapter_down): Sequential(
    (0): Linear(in_features=768, out_features=48, bias=True)
    (1): Activation_Function_Class(
      (f): ReLU()
    )
  )

In [ ]:
model.active_adapters

Stack[la_t]

In [ ]:
# currently active adapters
print("Active adapters:", model.active_adapters)

Active adapters: Stack[la_t]


In [ ]:
SFE_MODEL = model

In [ ]:
import torch
from transformers import DataCollatorForLanguageModeling

def test_mlm_with_adapter(text, adapter_name=None):
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"].clone().to(model.device)
    attention_mask = inputs["attention_mask"].clone().to(model.device)

    masked_indices = (input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

    if adapter_name:
        model.set_active_adapters(Stack(adapter_name))

    model.eval()

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
    # Move results back to CPU for token conversion
    input_ids_cpu = input_ids.cpu()
    for idx in masked_indices:
        predicted_token_id = torch.argmax(outputs.logits[0, idx], dim=-1).item()
        input_ids_cpu[0, idx] = predicted_token_id

    original_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
    predicted_tokens = tokenizer.convert_ids_to_tokens(input_ids_cpu.squeeze().tolist())
    original_text = tokenizer.convert_tokens_to_string(original_tokens)
    predicted_text = tokenizer.convert_tokens_to_string(predicted_tokens)
    return {
        "original_tokens": original_tokens,
        "predicted_tokens": predicted_tokens,
        "original_text": original_text,
        "predicted_text": predicted_text
    }

# Test
test_text = "ئۇيغۇر تىلى مېنىڭ مەدەنىي مىراثىم."
result = test_mlm_with_adapter(test_text, "la_s")

print("original_tokens:", result["original_tokens"])
print("predicted_tokens:", result["predicted_tokens"])
print("original_text:", result["original_text"])
print("predicted_text:", result["predicted_text"])


original_tokens: ['[CLS]', 'ئ', '##ۇ', '##ي', '##غ', '##ۇ', '##ر', 'ت', '##ى', '##لى', '[UNK]', '[UNK]', 'م', '##ى', '##را', '##ث', '##ى', '##م', '.', '[SEP]']
predicted_tokens: ['[CLS]', 'ئ', '##ۇ', '##ي', '##غ', '##ۇ', '##ر', 'ت', '##ى', '##لى', '[UNK]', '[UNK]', 'م', '##ى', '##را', '##ث', '##ى', '##م', '.', '[SEP]']
original_text: [CLS] ئۇيغۇر تىلى [UNK] [UNK] مىراثىم. [SEP]
predicted_text: [CLS] ئۇيغۇر تىلى [UNK] [UNK] مىراثىم. [SEP]


## AdapterFusionPLus

In [ ]:
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

class AdapterFusionPlus(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(2 * hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 2)
        )
    def forward(self, outputs_s, outputs_t):
        combined = torch.cat([outputs_s, outputs_t], dim=-1)  # [B, T, 2*H]
        raw_weights = self.mlp(combined)                      # [B, T, 2]
        weights = torch.softmax(raw_weights, dim=-1)          # [B, T, 2]

        w_s = weights[:, :, 0].unsqueeze(-1)  # [B, T, 1]
        w_t = weights[:, :, 1].unsqueeze(-1)  # [B, T, 1]

        fused_output = w_s * outputs_s + w_t * outputs_t      # [B, T, H]
        return fused_output

fusion_layer = AdapterFusionPlus(model.config.hidden_size)

def forward_with_fusion(input_ids):
    outputs_s = model(input_ids, adapter_names="la_s")
    outputs_t = model(input_ids, adapter_names="la_t")
    fused_output = fusion_layer(outputs_s.last_hidden_state, outputs_t.last_hidden_state)
    return fused_output

## POS

In [ ]:
from collections import defaultdict
import torch
import torch.nn as nn
import re
from transformers import AutoModelForTokenClassification, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn_crfsuite import CRF, metrics
from conllu import parse_incr
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder


def build_pos_vocab(conllu_data_path):
    pos_vocab = defaultdict(int)

    with open(conllu_data_path, "r") as f:
        for line in f:
            if line.startswith("#") or not line.strip():
                continue

            parts = line.strip().split("\t")

            if "-" in parts[0]:
                continue
            pos_tag = parts[3]
            pos_vocab[pos_tag] += 1

    pos_vocab["[PAD]"] = 0  # For padding
    pos_vocab["[UNK]"] = 0  # For unknown tags

    return pos_vocab


conllu_pth = '/content/ug_udt-ud-train.conllu'

all_pos_tags = []
with open(conllu_pth, "r") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split("\t")
        if "-" in parts[0]:
            continue
        all_pos_tags.append(parts[3])  # or parts[4]

le = LabelEncoder()
le.fit(all_pos_tags + ["[PAD]", "[UNK]"])

num_pos_tags = len(le.classes_)

pos_vocab = build_pos_vocab(conllu_pth)
num_pos_tags = len(pos_vocab)
print(f"Number of POS tags: {num_pos_tags}")

sorted_tags = sorted(
    [tag for tag in pos_vocab.keys() if tag not in ["[PAD]", "[UNK]"]],
    key=lambda x: pos_vocab[x],
    reverse=True
)

sorted_tags += ["[PAD]", "[UNK]"]

#  label2id mapping
pos_label2id = {tag: idx for idx, tag in enumerate(sorted_tags)}
id2pos_label = {idx: tag for tag, idx in pos_label2id.items()}

print(pos_label2id)


def load_conllu_data(file_path):
    """Loads CoNLL-U formatted data and extracts sentences with POS tags."""
    data_file = open(file_path, "r", encoding="utf-8")
    ud_treebank = []

    for tokenlist in parse_incr(data_file):
        tokens, tags = [], []
        for token in tokenlist:
            tokens.append(token["form"])
            tags.append(token["upostag"])
        ud_treebank.append((tokens, tags))

    return ud_treebank

conllu_pth = '/content/ug_udt-ud-train.conllu'
ud_treebank = load_conllu_data(conllu_pth)


def extract_features(sentence, index):
    """Extracts linguistic features from a given sentence at a specific index."""
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'is_alphanumeric': bool(re.match(r'^(?=.*[0-9]$)(?=.*[a-zA-Z])', sentence[index])),
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
    }

def transform_to_dataset(tagged_sentences):
    """Transforms tokenized sentences into feature sets and corresponding labels."""
    X, y = [], []
    for sentence, tags in tagged_sentences:
        X.append([extract_features(sentence, i) for i in range(len(sentence))])
        y.append(tags)
    return X, y


Number of POS tags: 12
{'NOUN': 0, 'PUNCT': 1, 'VERB': 2, 'NUM': 3, 'PRON': 4, 'ADJ': 5, 'PROPN': 6, 'INTJ': 7, 'ADV': 8, 'AUX': 9, '[PAD]': 10, '[UNK]': 11}


In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

class POSDataset(Dataset):
    """Custom dataset for POS tagging with BERT tokenizer."""
    def __init__(self, ud_treebank, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.features = []
        self.labels = []
        self.sentences = []

        all_tags = {tag for _, tags in ud_treebank for tag in tags}
        self.label_map = {tag: i for i, tag in enumerate(sorted(all_tags))}
        self.id2label = {i: tag for tag, i in self.label_map.items()}

        for tokens, tags in ud_treebank:
            self.sentences.append(tokens)
            inputs = tokenizer(
                tokens,
                is_split_into_words=True,
                padding="max_length",
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            )

            word_ids = inputs.word_ids()
            previous_word_id = None
            label_ids = []

            for word_id in word_ids:
                if word_id is None:
                    label_ids.append(-100)
                elif word_id != previous_word_id:
                    label_ids.append(self.label_map[tags[word_id]])
                else:
                    label_ids.append(-100)
                previous_word_id = word_id

            self.features.append({k: v.squeeze(0) for k, v in inputs.items()})
            self.labels.append(torch.tensor(label_ids))

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        readable_labels = [self.id2label[label.item()] for label in self.labels[idx] if label.item() != -100]

        return {
            "sentence": sentence,
            "input_ids": self.features[idx]["input_ids"],
            "attention_mask": self.features[idx]["attention_mask"],
            "labels_readable": readable_labels,
            "labels": self.labels[idx]
        }

dataset = POSDataset(ud_treebank, tokenizer)

sample = dataset[0]
print("Sentence:", " ".join(sample["sentence"]))
print("Labels:", sample["labels_readable"])


Sentence: نەشپۈت بەش يىلدا ، ئۆرۈك تۆت يىلدا مېۋە بېرىدۇ دېگەننى ئاڭلىمىغانمىدىڭ ؟
Labels: ['NOUN', 'NUM', 'NOUN', 'PUNCT', 'NOUN', 'NUM', 'NOUN', 'NOUN', 'VERB', 'VERB', 'VERB', 'PUNCT']


# POS USING AdapterFusionPlus

In [ ]:
class POSClassifier(nn.Module):
    """Final classifier for POS tagging."""
    def __init__(self, hidden_size, num_labels):
        super().__init__()
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, fused_output):
        return self.classifier(fused_output)

class POSFusionModel(nn.Module):
    """POS tagging model with AdapterFusion+."""
    def __init__(self, base_model, fusion_layer, classifier):
        super().__init__()
        self.base_model = base_model
        self.fusion_layer = fusion_layer
        self.classifier = classifier
        self.base_model.config.output_hidden_states = True

    def forward(self, input_ids, attention_mask):
        outputs_s = self.base_model(input_ids=input_ids, attention_mask=attention_mask, adapter_names="la_s")
        outputs_t = self.base_model(input_ids=input_ids, attention_mask=attention_mask, adapter_names="la_t")
        fused = self.fusion_layer(outputs_s.hidden_states[-1], outputs_t.hidden_states[-1])
        return self.classifier(fused)


def train_pos_model(model, dataloader, num_epochs=3, lr=1e-4):
    optimizer = AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch["input_ids"].cuda(), batch["attention_mask"].cuda(), batch["labels"].cuda()

            logits = model(input_ids, attention_mask)
            loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss / len(dataloader):.4f}")



def evaluate_pos_model(model, dataloader):
    model.eval()
    total_correct, total_tokens = 0, 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = (
                batch["input_ids"].cuda(),
                batch["attention_mask"].cuda(),
                batch["labels"].cuda(),
            )

            logits = model(input_ids, attention_mask)
            predictions = torch.argmax(logits, dim=-1)
            # print("labels",labels )
            # print("predictions", predictions)
            mask = labels != -100
            correct = (predictions == labels) & mask
            total_correct += correct.sum().item()
            total_tokens += mask.sum().item()
            y_true.extend(labels[mask].cpu().numpy().tolist())
            y_pred.extend(predictions[mask].cpu().numpy().tolist())

    print(f"POS Tagging Accuracy: {total_correct / total_tokens:.4f}")
    return y_true, y_pred

def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]

    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

    return {"input_ids": input_ids.cuda(), "attention_mask": attention_mask.cuda(), "labels": labels.cuda()}

train_dataset = POSDataset(load_conllu_data(conllu_pth), tokenizer)
test_dataset = POSDataset(load_conllu_data("/content/ug_udt-ud-dev.conllu"), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

fusion_layer = AdapterFusionPlus(768).cuda()
pos_classifier = POSClassifier(768, len(train_dataset.label_map)).cuda()
model1 =  SFE_MODEL
pos_model = POSFusionModel(model1, fusion_layer, pos_classifier).cuda()

train_pos_model(pos_model, train_loader, num_epochs=100)
y_true, y_pred = evaluate_pos_model(pos_model, test_loader)

Epoch 1/100 - Loss: 2.3731
Epoch 2/100 - Loss: 2.3246
Epoch 3/100 - Loss: 2.2854
Epoch 4/100 - Loss: 2.2377
Epoch 5/100 - Loss: 2.2047
Epoch 6/100 - Loss: 2.1608
Epoch 7/100 - Loss: 2.1269
Epoch 8/100 - Loss: 2.0858
Epoch 9/100 - Loss: 2.0492
Epoch 10/100 - Loss: 2.0222
Epoch 11/100 - Loss: 1.9909
Epoch 12/100 - Loss: 1.9627
Epoch 13/100 - Loss: 1.9421
Epoch 14/100 - Loss: 1.9120
Epoch 15/100 - Loss: 1.8956
Epoch 16/100 - Loss: 1.8666
Epoch 17/100 - Loss: 1.8506
Epoch 18/100 - Loss: 1.8291
Epoch 19/100 - Loss: 1.8015
Epoch 20/100 - Loss: 1.7922
Epoch 21/100 - Loss: 1.7759
Epoch 22/100 - Loss: 1.7619
Epoch 23/100 - Loss: 1.7509
Epoch 24/100 - Loss: 1.7345
Epoch 25/100 - Loss: 1.7185
Epoch 26/100 - Loss: 1.7060
Epoch 27/100 - Loss: 1.6931
Epoch 28/100 - Loss: 1.6952
Epoch 29/100 - Loss: 1.6605
Epoch 30/100 - Loss: 1.6669
Epoch 31/100 - Loss: 1.6470
Epoch 32/100 - Loss: 1.6418
Epoch 33/100 - Loss: 1.6342
Epoch 34/100 - Loss: 1.6245
Epoch 35/100 - Loss: 1.6138
Epoch 36/100 - Loss: 1.5991
E

In [ ]:
def predict_pos(sentence):
    inputs = tokenizer(sentence.split(), is_split_into_words=True, padding="max_length", truncation=True, return_tensors="pt").to("cuda")

    with torch.no_grad():
        logits = pos_model(inputs["input_ids"], inputs["attention_mask"])

    predictions = torch.argmax(logits, dim=-1).squeeze().tolist()
    pred_labels = [id2pos_label[i] for i in predictions if i != -100]

    return list(zip(sentence.split(), pred_labels))


test_sentence = "نەشپۈت بەش يىلدا، ئۆرۈك تۆت يىلدا مېۋە بېرىدۇ دېگەننى ئاڭلىمىغانمىدىڭ؟"
# test_sentence = "neshpüt besh yilda, örük töt yilda mëwe bëridu dëgenni anglimighanmiding?"
print(predict_pos(test_sentence))

[('نەشپۈت', 'ADV'), ('بەش', 'PRON'), ('يىلدا،', 'PRON'), ('ئۆرۈك', 'PRON'), ('تۆت', 'PRON'), ('يىلدا', 'PRON'), ('مېۋە', 'ADV'), ('بېرىدۇ', 'ADV'), ('دېگەننى', 'PRON'), ('ئاڭلىمىغانمىدىڭ؟', 'ADJ')]


In [ ]:
from sklearn.metrics import f1_score, classification_report

print("## AdapterFusion Model Evaluation ##")

f1 = f1_score(y_true, y_pred, average="weighted")
print("F1 score on Dev Data:", f1)

print("\nClass-wise scores:")
print(classification_report(y_true, y_pred, target_names=test_dataset.label_map.keys()))

## AdapterFusion Model Evaluation ##
F1 score on Dev Data: 0.08887595610242767

Class-wise scores:
              precision    recall  f1-score   support

         ADJ       0.00      0.00      0.00        16
         ADP       0.00      0.00      0.00         4
         ADV       0.00      0.00      0.00         4
         AUX       0.00      0.00      0.00         7
       CCONJ       0.01      1.00      0.02         1
        NOUN       0.00      0.00      0.00        36
         NUM       0.00      0.00      0.00         8
        PRON       0.00      0.00      0.00        18
       PUNCT       1.00      1.00      1.00        11
       SCONJ       0.00      0.00      0.00         1
        VERB       0.00      0.00      0.00        18

    accuracy                           0.10       124
   macro avg       0.09      0.18      0.09       124
weighted avg       0.09      0.10      0.09       124



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## POS USING AdapterFusion

In [ ]:
from adapters.composition import Fuse
from sklearn.metrics import f1_score, accuracy_score, classification_report

train_data = load_conllu_data(conllu_pth)
dev_data = load_conllu_data("/content/ug_udt-ud-test.conllu")

# POS label mapping
all_tags = sorted({tag for _, tags in train_data+dev_data for tag in tags})
id2label = {i: tag for i, tag in enumerate(all_tags)}
label2id = {tag: i for i, tag in id2label.items()}

# Tokenization and Dataset
# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

train_dataset = POSDataset(train_data, tokenizer)
dev_dataset = POSDataset(dev_data, tokenizer)

def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=2)
    true_labels = p.label_ids
    mask = true_labels != -100
    y_true = true_labels[mask].flatten()
    y_pred = predictions[mask].flatten()

    return {
        "f1": f1_score(y_true, y_pred, average="weighted"),
        "accuracy": accuracy_score(y_true, y_pred)
    }

In [ ]:
import numpy as np

model = AutoAdapterModel.from_pretrained("bert-base-multilingual-cased")

# Load two adapters
model.load_adapter("la_s", load_as="source_script")
model.load_adapter("la_t", load_as="transliterated")

adapter_setup = Fuse("source_script", "transliterated")
model.add_adapter_fusion(adapter_setup)

# POS classification head
model.add_tagging_head("pos_head", num_labels=len(id2label))

# Activate fusion
model.train_adapter_fusion(adapter_setup)

#  Training
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    logging_dir="./logs",
    output_dir="./pos_fusion",
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,2.664313,0.013478,0.030172
2,No log,2.826080,0.012737,0.030172
3,No log,2.952394,0.004940,0.021552
4,No log,3.037903,0.000771,0.017241
5,No log,3.065980,0.000802,0.017241
6,No log,3.044214,0.000884,0.017241
7,No log,3.053801,0.000907,0.017241
8,No log,3.073239,0.000867,0.017241
9,No log,3.087412,0.000857,0.017241
10,No log,3.088597,0.000867,0.017241


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=80, training_loss=1.6398483276367188, metrics={'train_runtime': 83.2067, 'train_samples_per_second': 1.923, 'train_steps_per_second': 0.961, 'total_flos': 13425909104640.0, 'train_loss': 1.6398483276367188, 'epoch': 10.0})

In [ ]:
# Evaluation and Prediction
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict_pos(sentence):
    tokenized = tokenizer(
        sentence.split(),
        is_split_into_words=True,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    ).to(device)

    with torch.no_grad():
        outputs = model(**tokenized)

    predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()
    word_ids = tokenized.word_ids(batch_index=0)

    pred_tags = []
    current_word = None
    for i, word_idx in enumerate(word_ids):
        if word_idx != current_word and word_idx is not None:
            pred_tags.append(id2label[predictions[i]])
            current_word = word_idx

    return list(zip(sentence.split(), pred_tags))

# test_sentence = "نەشپۈت بەش يىلدا، ئۆرۈك تۆت يىلدا مېۋە بېرىدۇ دېگەننى ئاڭلىمىغانمىدىڭ؟"
# test_sentence = "neshpüt besh yilda, örük töt yilda mëwe bëridu dëgenni anglimighanmiding ?"
test_sentence = "سەن شۇ چاققىچە تاقەت قىلىپ تۇرالامسەن!"
print(predict_pos(test_sentence))

# Evaluation Report
def get_true_pred(model, dataset):
    all_true = []
    all_pred = []
    model.eval()

    for item in dataset:
        inputs = {
            "input_ids": item["input_ids"].unsqueeze(0).to(device),
            "attention_mask": item["attention_mask"].unsqueeze(0).to(device)
        }

        with torch.no_grad():
            outputs = model(**inputs)

        preds = torch.argmax(outputs.logits, dim=-1).squeeze().cpu().tolist()
        labels = item["labels"].tolist()

        # mask ignoring -100 labels
        mask = [label != -100 for label in labels]
        all_true.extend([labels[i] for i, m in enumerate(mask) if m])
        all_pred.extend([preds[i] for i, m in enumerate(mask) if m])

    return all_true, all_pred

# predictions and labels
y_true, y_pred = get_true_pred(model, dev_dataset)

# classification report
print("## AdapterFusion Model Evaluation ##")
print("F1 score on Dev Data:", f1_score(y_true, y_pred, average="weighted"))
print("\nClass-wise scores:")
print(classification_report(
    [id2label[t] for t in y_true],
    [id2label[p] for p in y_pred],
    digits=3,
    zero_division=0
))

[('سەن', 'CCONJ'), ('شۇ', 'CCONJ'), ('چاققىچە', 'CCONJ'), ('تاقەت', 'CCONJ'), ('قىلىپ', 'PRON'), ('تۇرالامسەن!', 'PRON')]
## AdapterFusion Model Evaluation ##
F1 score on Dev Data: 0.0008674907829104316

Class-wise scores:
              precision    recall  f1-score   support

         ADJ      0.000     0.000     0.000        15
         ADP      0.000     0.000     0.000         9
         ADV      0.000     0.000     0.000         7
         AUX      0.000     0.000     0.000         7
       CCONJ      0.026     1.000     0.050         4
         DET      0.000     0.000     0.000         2
        INTJ      0.000     0.000     0.000        88
        NOUN      0.000     0.000     0.000         4
         NUM      0.000     0.000     0.000        12
        PRON      0.000     0.000     0.000        39
       PROPN      0.000     0.000     0.000        45

    accuracy                          0.017       232
   macro avg      0.002     0.091     0.005       232
weighted avg      0

# Dependency Parsing AdapterFusionPlus

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer, AdamW
from adapters import AutoAdapterModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class AdapterFusionPlus(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(2 * hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 2)
        )
    def forward(self, outputs_s, outputs_t):
        combined = torch.cat([outputs_s, outputs_t], dim=-1)   # [B, T, 2*H]
        raw_weights = self.mlp(combined)                       # [B, T, 2]
        weights = torch.softmax(raw_weights, dim=-1)           # [B, T, 2]
        w_s = weights[:, :, 0].unsqueeze(-1)                   # [B, T, 1]
        w_t = weights[:, :, 1].unsqueeze(-1)                   # [B, T, 1]
        fused_output = w_s * outputs_s + w_t * outputs_t       # [B, T, H]
        return fused_output

def create_dep_label_mapping(file_path):
    """
    Reads a CoNLL-U file and generates a unique mapping for dependency labels.
    """
    dep_labels_set = set()
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.startswith("#") and line.strip():
                parts = line.split("\t")
                if len(parts) == 10:
                    dep_labels_set.add(parts[7])
    dep_label2id = {label: idx for idx, label in enumerate(sorted(dep_labels_set))}
    dep_id2label = {idx: label for label, idx in dep_label2id.items()}
    return dep_label2id, dep_id2label

conllu_pth = "ug_udt-ud-train.conllu"
file_path = conllu_pth
dep_label2id, dep_id2label = create_dep_label_mapping(file_path)
NUM_DEP_LABELS = len(dep_label2id)
print(f"Dependency Labels Mapping: {dep_label2id}")


Dependency Labels Mapping: {'acl': 0, 'advcl': 1, 'advmod': 2, 'amod': 3, 'ccomp': 4, 'compound': 5, 'conj': 6, 'cop': 7, 'det': 8, 'discourse': 9, 'dislocated': 10, 'flat': 11, 'nmod': 12, 'nsubj': 13, 'nummod': 14, 'obj': 15, 'obl': 16, 'orphan': 17, 'parataxis': 18, 'punct': 19, 'root': 20, 'vocative': 21, 'xcomp': 22}


In [ ]:

class ConlluDataset(Dataset):
    def __init__(self, file_path, tokenizer, dep_label2id, max_length=128):
        self.sentences = self.load_conllu(file_path, dep_label2id)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def load_conllu(self, file_path, dep_label2id):

        sentences = []
        with open(file_path, "r", encoding="utf-8") as f:
            sentence = []
            for line in f:
                line = line.strip()
                if line == "":
                    if sentence:
                        sentences.append(sentence)
                        sentence = []
                elif not line.startswith("#"):
                    parts = line.split("\t")
                    if len(parts) == 10:
                        token_id, token, lemma, upos, xpos, feats, head, dep_rel, deps, misc = parts
                        label = dep_label2id.get(dep_rel, -1)
                        sentence.append((token, upos, int(head), label))
            if sentence:
                sentences.append(sentence)
        return sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        tokens = [word[0] for word in sentence]
        dep_heads = [word[2] for word in sentence]
        dep_labels = [word[3] for word in sentence]

        encoding = self.tokenizer(
            tokens,
            is_split_into_words=True,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            max_length=self.max_length,
            return_offsets_mapping=True
        )
        word_starts = torch.zeros(len(encoding["input_ids"][0]), dtype=torch.long)
        word_ids = encoding.word_ids()
        for i, word_idx in enumerate(word_ids):
            if word_idx is not None and (i == 0 or word_ids[i-1] != word_idx):
                word_starts[i] = 1

        padded_dep_labels = torch.full((self.max_length,), -1, dtype=torch.long)
        padded_dep_heads = torch.full((self.max_length,), -1, dtype=torch.long)
        seq_len = len(dep_labels)
        padded_dep_labels[:seq_len] = torch.tensor(dep_labels, dtype=torch.long)
        padded_dep_heads[:seq_len] = torch.tensor(dep_heads, dtype=torch.long)

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "dep_heads": padded_dep_heads,
            "dep_labels": padded_dep_labels,
            "word_starts": word_starts       # shape: [max_length]
        }

def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    dep_heads = torch.stack([item['dep_heads'] for item in batch])
    dep_labels = torch.stack([item['dep_labels'] for item in batch])
    word_starts = torch.stack([item['word_starts'] for item in batch])
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "dep_heads": dep_heads,
        "dep_labels": dep_labels,
        "word_starts": word_starts
    }


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

base_model = AutoAdapterModel.from_pretrained("bert-base-multilingual-cased").to(device)
base_model.load_adapter("la_s")
base_model.load_adapter("la_t")

class DependencyParser(nn.Module):

    def __init__(self, hidden_size, num_labels):
        super().__init__()
        self.classifier = nn.Linear(hidden_size, num_labels)
    def forward(self, fused_output):
        return self.classifier(fused_output)

class DependencyParserFusionModel(nn.Module):

    def __init__(self, base_model, fusion_layer, classifier):
        super().__init__()
        self.base_model = base_model
        self.fusion_layer = fusion_layer
        self.classifier = classifier
        self.base_model.config.output_hidden_states = True

    def forward(self, input_ids, attention_mask):
        # Activate and get outputs from the first adapter
        self.base_model.set_active_adapters(["la_s"])
        outputs_s = self.base_model(input_ids=input_ids, attention_mask=attention_mask)

        # Activate and get outputs from the second adapter
        self.base_model.set_active_adapters(["la_t"])
        outputs_t = self.base_model(input_ids=input_ids, attention_mask=attention_mask)

        # Fuse the outputs from both adapters
        fused = self.fusion_layer(outputs_s.hidden_states[-1], outputs_t.hidden_states[-1])
        # classification logits: shape [B, seq_len, NUM_DEP_LABELS]
        return self.classifier(fused)

fusion_layer = AdapterFusionPlus(base_model.config.hidden_size).to(device)
dependency_parser_classifier = DependencyParser(base_model.config.hidden_size, NUM_DEP_LABELS).to(device)
DepPar_model = DependencyParserFusionModel(base_model, fusion_layer, dependency_parser_classifier).to(device)


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def train_DepPar_model(model, dataloader, num_epochs=3, lr=1e-4):
    optimizer = AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=-1)
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["dep_labels"].to(device)
            logits = model(input_ids, attention_mask)
            loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss / len(dataloader):.4f}")

def evaluate_DepPar_model(model, dataloader):
    model.eval()
    total_correct, total_tokens = 0, 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["dep_labels"].to(device)
            logits = model(input_ids, attention_mask)
            predictions = torch.argmax(logits, dim=-1)
            mask = labels != -1
            correct = (predictions == labels) & mask
            total_correct += correct.sum().item()
            total_tokens += mask.sum().item()
            y_true.extend(labels[mask].cpu().tolist())
            y_pred.extend(predictions[mask].cpu().tolist())
    las = total_correct / total_tokens if total_tokens > 0 else 0
    print(f"Dependency Parser LAS: {las:.4f}")
    return y_true, y_pred

train_dataset = ConlluDataset(conllu_pth, tokenizer, dep_label2id)
val_dataset = ConlluDataset("/content/ug_udt-ud-test.conllu", tokenizer, dep_label2id)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

train_DepPar_model(DepPar_model, train_loader, num_epochs=10)
_ = evaluate_DepPar_model(DepPar_model, val_loader)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/adapters/composition.py:225: FutureWarning: Passing list objects for adapter activation is deprecated. Please use Stack or Fuse explicitly.
  warnings.warn(


Epoch 1/10 - Loss: 2.8163
Epoch 2/10 - Loss: 2.7207
Epoch 3/10 - Loss: 2.6777
Epoch 4/10 - Loss: 2.6333
Epoch 5/10 - Loss: 2.6665
Epoch 6/10 - Loss: 2.6523
Epoch 7/10 - Loss: 2.6224
Epoch 8/10 - Loss: 2.6460
Epoch 9/10 - Loss: 2.6841
Epoch 10/10 - Loss: 2.6443
Dependency Parser LAS: 0.2074


In [ ]:
def parse_sentence_dep_fusion(model, tokenizer, sentence, dep_id2label, max_length=128):

    original_tokens = sentence.strip().split()

    encoding = tokenizer(
        original_tokens,
        is_split_into_words=True,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        max_length=max_length,
        return_offsets_mapping=True
    )

    word_ids = encoding.word_ids(0)
    word_starts = torch.zeros(len(encoding["input_ids"][0]), dtype=torch.long)
    model_words = []
    previous_word_idx = None
    for pos, word_idx in enumerate(word_ids):
        if word_idx is None:
            continue
        if word_idx != previous_word_idx:
            word_starts[pos] = 1
            model_words.append(original_tokens[word_idx])
            previous_word_idx = word_idx

    word_starts = word_starts.unsqueeze(0)
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    model.eval()
    with torch.no_grad():
        logits = model(input_ids, attention_mask)  # shape: [1, seq_len, NUM_DEP_LABELS]
        predictions = torch.argmax(logits, dim=-1).squeeze(0).cpu().tolist()

    word_pred_labels = []
    for pos, word_idx in enumerate(word_ids):
        if word_idx is None:
            continue
        if pos == 0 or (word_ids[pos - 1] != word_idx):
            word_pred_labels.append(predictions[pos])

    simulated_heads = []
    for i in range(len(model_words)):
        if i == 0:
            simulated_heads.append("ROOT")
        else:
            simulated_heads.append(model_words[i - 1])

    print("Dependency Parsing Result:")
    print("OrigToken\tParsedToken\tPredicted Head\tDependency Label")
    for i in range(len(model_words)):
        token = original_tokens[i] if i < len(original_tokens) else "UNK"
        parsed_token = model_words[i]
        pred_label = dep_id2label.get(word_pred_labels[i], "UNKNOWN")
        pred_head = simulated_heads[i]
        print(f"{token}\t\t{parsed_token}\t\t{pred_head}\t\t\t{pred_label}")

test_sentence = "نەشپۈت بەش يىلدا، ئۆرۈك تۆت يىلدا مېۋە بېرىدۇ دېگەننى ئاڭلىمىغانمىدىڭ ؟"
parse_sentence_dep_fusion(DepPar_model, tokenizer, test_sentence, dep_id2label)


Dependency Parsing Result:
OrigToken	ParsedToken	Predicted Head	Dependency Label
نەشپۈت		نەشپۈت		ROOT			punct
بەش		بەش		نەشپۈت			punct
يىلدا،		يىلدا،		بەش			punct
ئۆرۈك		ئۆرۈك		يىلدا،			punct
تۆت		تۆت		ئۆرۈك			punct
يىلدا		يىلدا		تۆت			punct
مېۋە		مېۋە		يىلدا			punct
بېرىدۇ		بېرىدۇ		مېۋە			punct
دېگەننى		دېگەننى		بېرىدۇ			punct
ئاڭلىمىغانمىدىڭ		ئاڭلىمىغانمىدىڭ		دېگەننى			punct
؟		؟		ئاڭلىمىغانمىدىڭ			punct


# Dependency Parsing AdapterFusion




In [ ]:
import torch
from torch.utils.data import Dataset
from adapters.composition import Fuse
from sklearn.metrics import f1_score, accuracy_score
import numpy as np


def create_dep_label_mapping(file_path):
    mapping = {
        'acl': 0, 'advcl': 1, 'advmod': 2, 'amod': 3, 'ccomp': 4,
        'compound': 5, 'conj': 6, 'cop': 7, 'det': 8, 'discourse': 9,
        'dislocated': 10, 'flat': 11, 'nmod': 12, 'nsubj': 13, 'nummod': 14,
        'obj': 15, 'obl': 16, 'orphan': 17, 'parataxis': 18, 'punct': 19,
        'root': 20, 'vocative': 21, 'xcomp': 22
    }
    id2label = {v: k for k, v in mapping.items()}
    return mapping, id2label

def load_conllu_data(path, head_offset=64):
    data = []
    with open(path, encoding="utf-8") as f:
        lines = f.read().strip().split("\n")
    sentence_lines = []
    for line in lines:
        if line.strip() == "":
            if sentence_lines:
                data.append(parse_sentence(sentence_lines, head_offset))
                sentence_lines = []
        else:
            sentence_lines.append(line)
    if sentence_lines:
        data.append(parse_sentence(sentence_lines, head_offset))
    return data

def parse_sentence(lines, head_offset):
    sent_text = None
    tokens = []
    dep_heads = []
    dep_labels = []

    for line in lines:
        if line.startswith("# text ="):
            sent_text = line[len("# text ="):].strip()
            break
    for line in lines:
        if line.startswith("#"):
            continue
        parts = line.split("\t")
        if len(parts) < 8:
            continue
        if "-" in parts[0]:
            continue
        tokens.append(parts[1])
        head = int(parts[6])
        dep_heads.append(head)
        dep_labels.append(parts[7])
    if sent_text is None:
        sent_text = " ".join(tokens)
    return {
        "sentence": sent_text,
        "dep_heads": dep_heads,
        "dep_labels": dep_labels,
        "tokens": tokens
    }

class ConlluDataset(Dataset):
    def __init__(self, file_path, tokenizer, dep_label2id):
        self.data = load_conllu_data(file_path)
        self.tokenizer = tokenizer
        self.dep_label2id = dep_label2id

    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        example = self.data[idx]
        encoding = self.tokenizer(
            example["sentence"].split(),
            is_split_into_words=True,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=128
        )
        label_ids = [self.dep_label2id.get(label, -100) for label in example["dep_labels"]]

        max_length = encoding["input_ids"].shape[-1]
        label_tensor = torch.full((max_length,), -100, dtype=torch.long)
        label_tensor[:len(label_ids)] = torch.tensor(label_ids, dtype=torch.long)
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": label_tensor
        }

conllu_pth = "ug_udt-ud-train.conllu"
dev_conllu   = "ug_udt-ud-test.conllu"

# 1. Create dependency label mapping.
dep_label2id, dep_id2label = create_dep_label_mapping(conllu_pth)
print(f"Initial Dependency Labels Mapping: {dep_label2id}")

# 2. Initialize tokenizer and create datasets.
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
train_dataset = ConlluDataset(conllu_pth, tokenizer, dep_label2id)
val_dataset = ConlluDataset(dev_conllu, tokenizer, dep_label2id)

# 3. Update mapping based on training data to include unknown labels.
all_train_labels = set()
for example in train_dataset.data:
    for label in example["dep_labels"]:
        all_train_labels.add(label)
print("All labels in training set:", all_train_labels)
# Add any labels from the training set that are not in the mapping.
for label in all_train_labels:
    if label not in dep_label2id:
        dep_label2id[label] = len(dep_label2id)
dep_id2label = {v: k for k, v in dep_label2id.items()}
NUM_DEP_LABELS = len(dep_label2id)
print("Updated Dependency Labels Mapping:", dep_label2id)

# 4. Define compute_metrics.
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=2)
    true_labels = p.label_ids  # provided by collate_fn as "labels"
    mask = true_labels != -100  # ignore padding tokens
    y_true = true_labels[mask].flatten()
    y_pred = predictions[mask].flatten()
    return {
        "f1": f1_score(y_true, y_pred, average="weighted"),
        "accuracy": accuracy_score(y_true, y_pred)
    }

# 5. Model Setup with AdapterFusion.
model = AutoAdapterModel.from_pretrained("bert-base-multilingual-cased")
# Load two adapters.
model.load_adapter("la_s", load_as="source_script")
model.load_adapter("la_t", load_as="transliterated")
# Create and add adapter fusion.
adapter_setup = Fuse("source_script", "transliterated")
model.add_adapter_fusion(adapter_setup)
# Add dependency tagging head.
model.add_tagging_head("dep_head", num_labels=NUM_DEP_LABELS)
# Activate fusion and the tagging head.
model.train_adapter_fusion(adapter_setup)
model.set_active_adapters(adapter_setup)
if hasattr(model, "set_active_head"):
    model.set_active_head("dep_head")
else:
    model.active_head = "dep_head"

def collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.stack([item["labels"] for item in batch])
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    logging_dir="./logs",
    output_dir="./dep_fusion",
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

trainer.train()


Initial Dependency Labels Mapping: {'acl': 0, 'advcl': 1, 'advmod': 2, 'amod': 3, 'ccomp': 4, 'compound': 5, 'conj': 6, 'cop': 7, 'det': 8, 'discourse': 9, 'dislocated': 10, 'flat': 11, 'nmod': 12, 'nsubj': 13, 'nummod': 14, 'obj': 15, 'obl': 16, 'orphan': 17, 'parataxis': 18, 'punct': 19, 'root': 20, 'vocative': 21, 'xcomp': 22}
All labels in training set: {'acl', 'punct', 'nmod', 'xcomp', 'cop', 'conj', 'nummod', 'advcl', 'advmod', 'obl', 'discourse', 'dislocated', 'ccomp', 'flat', 'parataxis', 'root', 'vocative', 'orphan', 'compound', 'amod', 'obj', 'det', 'nsubj'}
Updated Dependency Labels Mapping: {'acl': 0, 'advcl': 1, 'advmod': 2, 'amod': 3, 'ccomp': 4, 'compound': 5, 'conj': 6, 'cop': 7, 'det': 8, 'discourse': 9, 'dislocated': 10, 'flat': 11, 'nmod': 12, 'nsubj': 13, 'nummod': 14, 'obj': 15, 'obl': 16, 'orphan': 17, 'parataxis': 18, 'punct': 19, 'root': 20, 'vocative': 21, 'xcomp': 22}


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,2.899213,0.077638,0.170213
2,No log,2.782184,0.070699,0.202128
3,No log,2.745335,0.071281,0.207447
4,No log,2.732247,0.071281,0.207447
5,No log,2.725943,0.071281,0.207447
6,No log,2.722145,0.070071,0.202128
7,No log,2.722893,0.070384,0.202128
8,No log,2.726014,0.070071,0.202128
9,No log,2.724043,0.070384,0.202128
10,No log,2.723901,0.070384,0.202128


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=80, training_loss=2.650547981262207, metrics={'train_runtime': 76.9272, 'train_samples_per_second': 2.08, 'train_steps_per_second': 1.04, 'total_flos': 13426854051840.0, 'train_loss': 2.650547981262207, 'epoch': 10.0})

In [ ]:
def infer_dependency_parsing(sentence, tokenizer, model, dep_id2label, max_length=128):
    encoding = tokenizer(
        sentence.split(),
        is_split_into_words=True,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    input_ids = encoding["input_ids"].to(model.device)
    attention_mask = encoding["attention_mask"].to(model.device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=-1)[0]

    word_ids = encoding.word_ids()
    predicted_labels = {}
    previous_word_idx = None
    for idx, word_idx in enumerate(word_ids):
        if word_idx is None:
            continue
        if word_idx != previous_word_idx:
            label_id = predictions[idx].item()
            predicted_labels[word_idx] = dep_id2label.get(label_id, "UNK")
            previous_word_idx = word_idx

    words = sentence.split()
    results = []
    for i, word in enumerate(words):
        orig_token = word
        parsed_token = word
        pred_head = "ROOT" if i == 0 else words[i-1]
        dep_label = predicted_labels.get(i, "UNK")
        results.append((orig_token, parsed_token, pred_head, dep_label))
    return results

sample_sentence = "نەشپۈت بەش يىلدا، ئۆرۈك تۆت يىلدا مېۋە بېرىدۇ دېگەننى ئاڭلىمىغانمىدىڭ ؟"
results = infer_dependency_parsing(sample_sentence, tokenizer, model, dep_id2label)

print("Dependency Parsing Result:")
print("OrigToken\t\tParsedToken\t\tPredicted Head\t\tDependency Label")
for orig, parsed, head, label in results:
    print(f"{orig}\t\t{parsed}\t\t{head}\t\t{label}")


Dependency Parsing Result:
OrigToken		ParsedToken		Predicted Head		Dependency Label
نەشپۈت		نەشپۈت		ROOT		punct
بەش		بەش		نەشپۈت		punct
يىلدا،		يىلدا،		بەش		punct
ئۆرۈك		ئۆرۈك		يىلدا،		punct
تۆت		تۆت		ئۆرۈك		punct
يىلدا		يىلدا		تۆت		punct
مېۋە		مېۋە		يىلدا		punct
بېرىدۇ		بېرىدۇ		مېۋە		punct
دېگەننى		دېگەننى		بېرىدۇ		punct
ئاڭلىمىغانمىدىڭ		ئاڭلىمىغانمىدىڭ		دېگەننى		punct
؟		؟		ئاڭلىمىغانمىدىڭ		punct


# Unipelt POS tagging

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForTokenClassification, AutoTokenizer
from peft import LoraConfig, PrefixTuningConfig, get_peft_model

def load_conllu_data(file_path):
    """Loads CoNLL-U formatted data and extracts (tokens, upostags)."""
    ud_treebank = []
    with open(file_path, "r", encoding="utf-8") as f:
        for tokenlist in parse_incr(f):
            tokens, tags = [], []
            for token in tokenlist:
                tokens.append(token["form"])
                tags.append(token["upostag"])
            ud_treebank.append((tokens, tags))
    return ud_treebank

class POSDataset(torch.utils.data.Dataset):
    def __init__(self, ud_treebank, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.max_length = max_length
        all_tags = {t for _, tags in ud_treebank for t in tags}
        self.label_map = {tag: i for i, tag in enumerate(sorted(all_tags))}
        self.id2label = {i: tag for tag, i in self.label_map.items()}
        self.features, self.labels = [], []

        for tokens, tags in ud_treebank:
            enc = tokenizer(
                tokens,
                is_split_into_words=True,
                padding="max_length",
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            )
            word_ids = enc.word_ids()
            label_ids = []
            prev_word = None

            for wid in word_ids:
                if wid is None:
                    label_ids.append(-100)
                elif wid != prev_word:
                    label_ids.append(self.label_map[tags[wid]])
                else:
                    label_ids.append(-100)
                prev_word = wid

            self.features.append({k: v.squeeze(0) for k, v in enc.items()})
            self.labels.append(torch.tensor(label_ids))

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return {
            "input_ids": self.features[idx]["input_ids"],
            "attention_mask": self.features[idx]["attention_mask"],
            "labels": self.labels[idx]
        }

def collate_fn(batch):
    return {
        "input_ids": torch.stack([b["input_ids"] for b in batch]),
        "attention_mask": torch.stack([b["attention_mask"] for b in batch]),
        "labels": torch.stack([b["labels"] for b in batch])
    }
class UniPELTModel(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.base_model = AutoModelForTokenClassification.from_pretrained(
            "bert-base-multilingual-cased",
            num_labels=num_labels,
            ignore_mismatched_sizes=True
        )

        # Create separate PEFT models
        self.lora_model = self._create_lora_model()
        self.prefix_model = self._create_prefix_model()
        # self.base_model.add_adapter("la_t", la_t_config)
        # self.base_model.add_adapter("la_s", la_s_config)
        # Fusion components
        self.fusion = nn.Sequential(
            nn.Linear(2 * self.base_model.config.hidden_size, 256),
            nn.ReLU(),
            nn.Linear(256, self.base_model.config.hidden_size)
        )
        # Final classifier
        self.classifier = nn.Linear(
            self.base_model.config.hidden_size,
            num_labels
        )

    def _create_lora_model(self):
        config = LoraConfig(
            r=8,
            lora_alpha=32,
            target_modules=["query", "value"],
            bias="none",
            task_type="TOKEN_CLS"
        )
        return get_peft_model(self.base_model, config)

    def _create_prefix_model(self):
        config = PrefixTuningConfig(
            num_virtual_tokens=20,
            task_type="TOKEN_CLS"
        )
        return get_peft_model(self.base_model, config)

    def forward(self, input_ids, attention_mask):
        # Get outputs from both PEFT models
        lora_output = self.lora_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )

        prefix_output = self.prefix_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )

        # Fusion of last hidden states
        fused = self.fusion(torch.cat([
            lora_output.hidden_states[-1],
            prefix_output.hidden_states[-1]
        ], dim=-1))

        return self.classifier(fused)

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
train_data = load_conllu_data("/content/ug_udt-ud-train.conllu")
train_dataset = POSDataset(train_data, tokenizer)

# Create model
model = UniPELTModel(num_labels=len(train_dataset.label_map)).cuda()

# Check trainable parameters
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 974868


In [ ]:
!git clone https://github.com/morningmoni/UniPELT.git
%cd UniPELT
!pip install -r requirements.txt
# 1) Make sure PEFT is installed
!pip install peft


Cloning into 'UniPELT'...
remote: Enumerating objects: 531, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 531 (delta 4), reused 0 (delta 0), pack-reused 518 (from 1)
Receiving objects: 100% (531/531), 1.59 MiB | 3.65 MiB/s, done.
Resolving deltas: 100% (209/209), done.
/content/UniPELT
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
%cd UniPELT

/content/UniPELT


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModelForTokenClassification, AutoTokenizer
from peft import LoraConfig, PrefixTuningConfig, get_peft_model

class UniPELTModel(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        # Base model
        self.base_model = AutoModelForTokenClassification.from_pretrained(
            "bert-base-multilingual-cased",
            num_labels=num_labels,
            ignore_mismatched_sizes=True
        )

        # Freeze base model
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Add LoRA
        lora_config = LoraConfig(
            r=8,
            lora_alpha=32,
            target_modules=["query", "value"],
            bias="none",
            task_type="TOKEN_CLS",
        )
        self.lora_model = get_peft_model(self.base_model, lora_config)

        # Add Prefix Tuning
        prefix_config = PrefixTuningConfig(
            num_virtual_tokens=20,
            task_type="TOKEN_CLS",
        )
        self.prefix_model = get_peft_model(self.base_model, prefix_config)

        # Fusion components
        self.gate = nn.Sequential(
            nn.Linear(2 * self.base_model.config.hidden_size, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
            nn.Softmax(dim=-1)
        )
        self.classifier = nn.Linear(
            self.base_model.config.hidden_size,
            num_labels
        )

    def forward(self, input_ids, attention_mask):
        # Get outputs from both PEFT models
        lora_output = self.lora_model.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )

        prefix_output = self.prefix_model.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )

        # Get last hidden states
        h_lora = lora_output.hidden_states[-1]
        h_prefix = prefix_output.hidden_states[-1]

        # Compute gating weights
        gate_weights = self.gate(torch.cat([h_lora, h_prefix], dim=-1))
        fused = gate_weights[:,:,0:1] * h_lora + gate_weights[:,:,1:2] * h_prefix

        return self.classifier(fused)

# Initialize components
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
train_data = load_conllu_data("/content/ug_udt-ud-train.conllu")


train_dataset = POSDataset(train_data, tokenizer)

train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=lambda batch: {
        'input_ids': torch.stack([x['input_ids'] for x in batch]),
        'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
        'labels': torch.stack([x['labels'] for x in batch])
    }
)

# Create model
model = UniPELTModel(num_labels=len(train_dataset.label_map)).cuda()

# Training setup
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4,
    weight_decay=0.01
)

# Training loop
for epoch in range(30):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs = {k: v.cuda() for k, v in batch.items()}

        optimizer.zero_grad()
        logits = model(inputs["input_ids"], inputs["attention_mask"])

        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            inputs["labels"].view(-1),
            ignore_index=-100
        )

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.cuda() for k, v in batch.items()}
        logits = model(inputs["input_ids"], inputs["attention_mask"])
        preds = torch.argmax(logits, dim=-1)

        mask = inputs["labels"] != -100
        correct += (preds[mask] == inputs["labels"][mask]).sum().item()
        total += mask.sum().item()

print(f"Validation Accuracy: {correct/total:.4f}")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 Loss: 2.3726
Epoch 2 Loss: 2.2726
Epoch 3 Loss: 2.1711
Epoch 4 Loss: 2.1106
Epoch 5 Loss: 2.0550
Epoch 6 Loss: 1.9820
Epoch 7 Loss: 1.9213
Epoch 8 Loss: 1.8696
Epoch 9 Loss: 1.8251
Epoch 10 Loss: 1.7668
Epoch 11 Loss: 1.7376
Epoch 12 Loss: 1.7326
Epoch 13 Loss: 1.6895
Epoch 14 Loss: 1.6791
Epoch 15 Loss: 1.6125
Epoch 16 Loss: 1.6097
Epoch 17 Loss: 1.5775
Epoch 18 Loss: 1.5642
Epoch 19 Loss: 1.5448
Epoch 20 Loss: 1.5044
Epoch 21 Loss: 1.4973
Epoch 22 Loss: 1.4730
Epoch 23 Loss: 1.4683
Epoch 24 Loss: 1.4503
Epoch 25 Loss: 1.4317
Epoch 26 Loss: 1.4140
Epoch 27 Loss: 1.3920
Epoch 28 Loss: 1.3930
Epoch 29 Loss: 1.3707
Epoch 30 Loss: 1.3839
Validation Accuracy: 0.0968


In [ ]:
def predict_pos(sentence, model, tokenizer, id2label):
    # Preprocess and tokenize
    words = sentence.split()

    # Tokenize with word IDs
    encodings = tokenizer(
        words,
        is_split_into_words=True,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    )

    # Get predictions
    with torch.no_grad():
        logits = model(
            encodings["input_ids"].to(device),
            encodings["attention_mask"].to(device)
        )

    # Convert to predictions
    predictions = logits.argmax(-1).squeeze().cpu().numpy()

    # Align with original words
    word_ids = encodings.word_ids()
    pos_tags = []
    current_word = None

    for idx, word_id in enumerate(word_ids):
        if word_id is None or word_id == current_word:
            continue
        current_word = word_id
        pos_tags.append((words[word_id], id2label[predictions[idx]]))

    return pos_tags

# Usage
test_sentence = "نەشپۈت بەش يىلدا، ئۆرۈك تۆت يىلدا مېۋە بېرىدۇ دېگەننى ئاڭلىمىغانمىدىڭ؟"
predictions = predict_pos(test_sentence, model, tokenizer, train_dataset.id2label)

# Print formatted results
for word, tag in predictions:
    print(f"{word:20} -> {tag}")

نەشپۈت               -> NOUN
بەش                  -> NOUN
يىلدا،               -> NOUN
ئۆرۈك                -> NOUN
تۆت                  -> NOUN
يىلدا                -> NOUN
مېۋە                 -> NOUN
بېرىدۇ               -> NOUN
دېگەننى              -> NOUN
ئاڭلىمىغانمىدىڭ؟     -> NOUN


# Dependecy Parsing Unipelt

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from peft import (
    LoraConfig,
    PrefixTuningConfig,
    get_peft_model,
    TaskType
)

# 1. Fix Dataset Class to Ensure Correct Keys
class ConlluDataset(Dataset):

    def __init__(self, file_path, tokenizer, dep_label2id, max_length=128):
        self.sentences = self.load_conllu(file_path, dep_label2id)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.dep_label2id = dep_label2id
    def __len__(self):  # ADD THIS METHOD
        return len(self.sentences)
    def load_conllu(self, file_path, dep_label2id):

        sentences = []
        with open(file_path, "r", encoding="utf-8") as f:
            sentence = []
            for line in f:
                line = line.strip()
                if line == "":
                    if sentence:
                        sentences.append(sentence)
                        sentence = []
                elif not line.startswith("#"):
                    parts = line.split("\t")
                    if len(parts) == 10:
                        token_id, token, lemma, upos, xpos, feats, head, dep_rel, deps, misc = parts
                        label = dep_label2id.get(dep_rel, -1)
                        sentence.append((token, upos, int(head), label))
            if sentence:
                sentences.append(sentence)
        return sentences

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        tokens = [word[0] for word in sentence]
        dep_heads = [word[2] for word in sentence]
        dep_labels = [word[3] for word in sentence]

        encoding = self.tokenizer(
            tokens,
            is_split_into_words=True,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
            return_attention_mask=True,
            return_offsets_mapping=True  # Correct parameter
        )

        # Get word IDs from the encoding
        word_ids = encoding.word_ids(batch_index=0)

        # Align dependencies
        aligned_heads = []
        aligned_labels = []
        current_word_idx = -1

        for wid in word_ids:
            if wid is None:
                aligned_heads.append(-1)
                aligned_labels.append(-1)
            elif wid != current_word_idx:
                current_word_idx = wid
                aligned_heads.append(dep_heads[wid])
                aligned_labels.append(dep_labels[wid])
            else:
                aligned_heads.append(-1)
                aligned_labels.append(-1)

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "dep_heads": torch.tensor(aligned_heads, dtype=torch.long),
            "dep_labels": torch.tensor(aligned_labels, dtype=torch.long)
        }

# 2. Verify Data Loading
def debug_dataset():
    sample = ConlluDataset(conllu_pth, tokenizer, dep_label2id)[0]
    print("Sample keys:", sample.keys())  # Should show dep_heads and dep_labels
    print("Dep heads shape:", sample["dep_heads"].shape)
    print("Dep labels shape:", sample["dep_labels"].shape)

debug_dataset()

# 3. Fixed Collate Function
def collate_fn(batch):
    return {
        "input_ids": torch.stack([item["input_ids"] for item in batch]),
        "attention_mask": torch.stack([item["attention_mask"] for item in batch]),
        "dep_heads": torch.stack([item["dep_heads"] for item in batch]),
        "dep_labels": torch.stack([item["dep_labels"] for item in batch])
    }

# 4. Initialize Data Loaders
train_loader = DataLoader(
    ConlluDataset(conllu_pth, tokenizer, dep_label2id),
    batch_size=4,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    ConlluDataset("/content/ug_udt-ud-test.conllu", tokenizer, dep_label2id),
    batch_size=4,
    collate_fn=collate_fn
)

class UniPELTDependencyParser(nn.Module):
    def __init__(self, num_dep_labels, max_seq_length=128):
        super().__init__()
        self.max_seq_length = max_seq_length

        # Base model
        self.base_model = AutoModel.from_pretrained("bert-base-multilingual-cased")

        # Freeze base model
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Adapter configurations
        lora_config = LoraConfig(
            r=8,
            lora_alpha=32,
            target_modules=["query", "value"],
            bias="none",
            task_type=TaskType.FEATURE_EXTRACTION
        )

        prefix_config = PrefixTuningConfig(
            num_virtual_tokens=20,
            task_type=TaskType.FEATURE_EXTRACTION
        )

        # Create PEFT models
        self.lora_model = get_peft_model(self.base_model, lora_config)
        self.prefix_model = get_peft_model(self.base_model, prefix_config)

        # Fusion components
        self.fusion_gate = nn.Sequential(
            nn.Linear(2 * self.base_model.config.hidden_size, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
            nn.Softmax(dim=-1)
        )

        # Dependency parsing heads
        self.head_predictor = nn.Linear(
            self.base_model.config.hidden_size,
            self.max_seq_length  # Now outputs scores for each position
        )
        self.label_predictor = nn.Linear(
            self.base_model.config.hidden_size,
            num_dep_labels
        )

    def forward(self, input_ids, attention_mask):
        # Get hidden states from both adapters
        lora_output = self.lora_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )

        prefix_output = self.prefix_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )

        # Fusion of last hidden states
        h_lora = lora_output.hidden_states[-1]
        h_prefix = prefix_output.hidden_states[-1]
        gate_weights = self.fusion_gate(torch.cat([h_lora, h_prefix], dim=-1))
        fused = gate_weights[:,:,0:1] * h_lora + gate_weights[:,:,1:2] * h_prefix

        # Predict dependencies
        head_scores = self.head_predictor(fused)  # [batch_size, seq_len, max_seq_length]
        label_logits = self.label_predictor(fused)  # [batch_size, seq_len, num_labels]

        return head_scores, label_logits

# Modified training function
def train_dep_parser(model, dataloader, num_epochs=3, lr=1e-4):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            dep_heads = batch["dep_heads"].to(device)
            dep_labels = batch["dep_labels"].to(device)

            # Forward pass
            head_scores, label_logits = model(input_ids, attention_mask)

            # Calculate losses
            head_loss = F.cross_entropy(
                head_scores.view(-1, model.max_seq_length),
                dep_heads.view(-1),
                ignore_index=-1
            )
            label_loss = F.cross_entropy(
                label_logits.view(-1, label_logits.size(-1)),
                dep_labels.view(-1),
                ignore_index=-1
            )

            total_loss = head_loss + label_loss
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += total_loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader):.4f}")
# Modified evaluation function
def evaluate_dep_parser(model, dataloader):
    model.eval()
    total_correct = 0
    total_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            dep_heads = batch["dep_heads"].to(device)
            dep_labels = batch["dep_labels"].to(device)

            head_scores, label_logits = model(input_ids, attention_mask)
            head_preds = torch.argmax(head_scores, dim=-1)
            label_preds = torch.argmax(label_logits, dim=-1)

            # Mask for valid tokens
            mask = (dep_labels != -1) & (dep_heads != -1)

            # Calculate correct predictions
            correct = ((head_preds == dep_heads) &
                      (label_preds == dep_labels) &
                      mask).sum().item()

            total_correct += correct
            total_tokens += mask.sum().item()

    las = total_correct / total_tokens if total_tokens > 0 else 0
    print(f"Labeled Attachment Score (LAS): {las:.4f}")
    return las

# Initialize model
dep_label2id, dep_id2label = create_dep_label_mapping(conllu_pth)
model = UniPELTDependencyParser(num_dep_labels=len(dep_label2id)).to(device)

# Training
# Now you can train normally
# train_dep_parser(model, train_loader)
train_dep_parser(model, train_loader, num_epochs=30)
evaluate_dep_parser(model, val_loader)

# Modified parsing function
def parse_sentence(text, model, tokenizer, dep_id2label):
    tokens = text.split()

    # Tokenize
    encoding = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    )

    # Get predictions
    head_scores, label_logits = model(
        encoding["input_ids"].to(device),
        encoding["attention_mask"].to(device)
    )

    # Convert to predictions
    head_preds = torch.argmax(head_scores, dim=-1).squeeze().cpu().numpy()
    label_preds = torch.argmax(label_logits, dim=-1).squeeze().cpu().numpy()

    # Align with original words
    word_ids = encoding.word_ids()
    predictions = []
    current_word = None

    for idx, word_id in enumerate(word_ids):
        if word_id is None or word_id == current_word:
            continue
        current_word = word_id
        predictions.append((
            tokens[word_id],
            head_preds[idx],
            dep_id2label[label_preds[idx]]
        ))

    # Print results
    print("Token\tHead\tDependency")
    for token, head, dep in predictions:
        print(f"{token}\t{head}\t{dep}")

# Example usage
test_sentence = "نەشپۈت بەش يىلدا، ئۆرۈك تۆت يىلدا مېۋە بېرىدۇ دېگەننى ئاڭلىمىغانمىدىڭ؟"
parse_sentence(test_sentence, model, tokenizer, dep_id2label)

Sample keys: dict_keys(['input_ids', 'attention_mask', 'dep_heads', 'dep_labels'])
Dep heads shape: torch.Size([128])
Dep labels shape: torch.Size([128])
Epoch 1 Loss: 3.9855
Epoch 2 Loss: 3.8642
Epoch 3 Loss: 3.8154
Epoch 4 Loss: 3.7011
Epoch 5 Loss: 3.6090
Epoch 6 Loss: 3.5024
Epoch 7 Loss: 3.3902
Epoch 8 Loss: 3.3119
Epoch 9 Loss: 3.2340
Epoch 10 Loss: 3.1786
Epoch 11 Loss: 3.1480
Epoch 12 Loss: 3.0740
Epoch 13 Loss: 3.0331
Epoch 14 Loss: 2.9296
Epoch 15 Loss: 2.8161
Epoch 16 Loss: 2.8476
Epoch 17 Loss: 2.7139
Epoch 18 Loss: 2.6879
Epoch 19 Loss: 2.7496
Epoch 20 Loss: 2.6257
Epoch 21 Loss: 2.6277
Epoch 22 Loss: 2.5603
Epoch 23 Loss: 2.5134
Epoch 24 Loss: 2.4888
Epoch 25 Loss: 2.5082
Epoch 26 Loss: 2.4459
Epoch 27 Loss: 2.4104
Epoch 28 Loss: 2.5026
Epoch 29 Loss: 2.3165
Epoch 30 Loss: 2.3688
Labeled Attachment Score (LAS): 0.0106
Token	Head	Dependency
نەشپۈت	9	nsubj
بەش	9	nsubj
يىلدا،	9	punct
ئۆرۈك	9	nsubj
تۆت	9	nummod
يىلدا	9	punct
مېۋە	9	punct
بېرىدۇ	9	punct
دېگەننى	9	nsubj
ئاڭلىمى

# Unipelt plus pos tagging

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForTokenClassification, AutoTokenizer
from peft import LoraConfig, PrefixTuningConfig, TaskType

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
class POSDataset(torch.utils.data.Dataset):
    def __init__(self, ud_treebank, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_map = {}
        self.id2label = {}
        self.features = []
        self.labels = []

        # Build label map
        all_tags = {t for _, tags in ud_treebank for t in tags}
        self.label_map = {tag: i for i, tag in enumerate(sorted(all_tags))}
        self.id2label = {v: k for k, v in self.label_map.items()}

        # Process sentences
        for tokens, tags in ud_treebank:
            encoding = tokenizer(
                tokens,
                is_split_into_words=True,
                padding="max_length",
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            )

            # Align labels with subword tokens
            word_ids = encoding.word_ids()
            label_ids = []
            current_word = None

            for wid in word_ids:
                if wid is None:  # Special tokens
                    label_ids.append(-100)
                elif wid != current_word:  # Start of new word
                    label_ids.append(self.label_map[tags[wid]])
                    current_word = wid
                else:  # Subword continuation
                    label_ids.append(-100)

            self.features.append({
                "input_ids": encoding["input_ids"].squeeze(0),
                "attention_mask": encoding["attention_mask"].squeeze(0)
            })
            self.labels.append(torch.tensor(label_ids))

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return {
            "input_ids": self.features[idx]["input_ids"],
            "attention_mask": self.features[idx]["attention_mask"],
            "labels": self.labels[idx]
        }

# Initialize dataset properly
# train_dataset = POSDataset(train_data, tokenizer)
def pos_collate_fn(batch):
    return {
        "input_ids": torch.stack([item["input_ids"] for item in batch]),
        "attention_mask": torch.stack([item["attention_mask"] for item in batch]),
        "labels": torch.stack([item["labels"] for item in batch])
    }

# 3. Initialize Data Loaders with Correct Collate Function
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=pos_collate_fn
)

test_loader = DataLoader(
    test_dataset,  # You'll need to create this
    batch_size=4,
    collate_fn=pos_collate_fn
)

class UniPELTFusionPlusPOS(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        # Shared base model
        self.base_model = AutoModelForTokenClassification.from_pretrained(
            "bert-base-multilingual-cased",
            ignore_mismatched_sizes=True
        )

        # Freeze base model
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Initialize adapters for each script
        self.script_models = nn.ModuleDict({
            script: nn.ModuleDict({
                'lora': get_peft_model(
                    AutoModelForTokenClassification.from_pretrained(
                        "bert-base-multilingual-cased",
                        num_labels=num_labels,
                        ignore_mismatched_sizes=True
                    ),
                    LoraConfig(
                        r=8,
                        lora_alpha=32,
                        target_modules=["query", "value"],
                        bias="none",
                        task_type=TaskType.TOKEN_CLS
                    )
                ),
                'prefix': get_peft_model(
                    AutoModelForTokenClassification.from_pretrained(
                        "bert-base-multilingual-cased",
                        num_labels=num_labels,
                        ignore_mismatched_sizes=True
                    ),
                    PrefixTuningConfig(
                        num_virtual_tokens=20,
                        task_type=TaskType.TOKEN_CLS
                    )
                )
            }) for script in ['s', 't']
        })

        # Fusion components
        self.fusion_gate = nn.Sequential(
            nn.Linear(2 * self.base_model.config.hidden_size, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Linear(256, 2),
            nn.Softmax(dim=-1)
        )

        # Final classifier
        self.classifier = nn.Linear(self.base_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        script_outputs = []

        # Get outputs for each script configuration
        for script in ['s', 't']:
            # Get LoRA output
            lora_out = self.script_models[script]['lora'](
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True
            ).hidden_states[-1]

            # Get Prefix output
            prefix_out = self.script_models[script]['prefix'](
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True
            ).hidden_states[-1]

            # Store combined script representation
            script_outputs.append(lora_out + prefix_out)

        # Fusion with residual connection
        combined = torch.cat(script_outputs, dim=-1)
        weights = self.fusion_gate(combined)
        fused = (weights[:,:,0:1] * script_outputs[0] +
                weights[:,:,1:2] * script_outputs[1] +
                0.5 * (script_outputs[0] + script_outputs[1]))

        return self.classifier(fused)

# Initialize components
# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
# train_data = load_conllu_data("/content/ug_udt-ud-train.conllu")
# train_dataset = POSDataset(train_data, tokenizer)

# # Create model
# model = UniPELTFusionPlusPOS(num_labels=len(train_dataset.label_map)).cuda()

# Enhanced training with gradient clipping
def train_pos_model(model, dataloader, num_epochs=30):
    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=2e-5,
        weight_decay=0.01
    )

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in dataloader:
            inputs = {k: v.cuda() for k, v in batch.items()}

            optimizer.zero_grad()
            logits = model(inputs["input_ids"], inputs["attention_mask"])

            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                inputs["labels"].view(-1),
                ignore_index=-100
            )

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader):.4f}")

def create_label_mapping(train_path, test_path=None):
    """Create label mapping considering both train and test data"""
    labels = set()

    # Process training data
    with open(train_path, "r", encoding="utf-8") as f:
        for line in f:
            if "\t" in line:
                parts = line.strip().split("\t")
                if len(parts) >= 4:
                    labels.add(parts[3])  # UPOS is in 4th column

    # Process test data if provided
    if test_path:
        with open(test_path, "r", encoding="utf-8") as f:
            for line in f:
                if "\t" in line:
                    parts = line.strip().split("\t")
                    if len(parts) >= 4:
                        labels.add(parts[3])

    # Create mappings
    label2id = {tag: idx for idx, tag in enumerate(sorted(labels))}
    id2label = {idx: tag for tag, idx in label2id.items()}
    return label2id, id2label

# Initialize with both train and test data
label2id, id2label = create_label_mapping(
    "/content/ug_udt-ud-train.conllu",
    "/content/ug_udt-ud-test.conllu"
)
# Enhanced evaluation with F1 score
def evaluate_pos(model, dataloader, id2label):
    model.eval()
    total_correct = 0
    total_tokens = 0
    class_counts = {label: {'tp': 0, 'fp': 0, 'fn': 0} for label in id2label.values()}

    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.cuda() for k, v in batch.items()}
            logits = model(inputs["input_ids"], inputs["attention_mask"])
            preds = torch.argmax(logits, dim=-1)

            mask = inputs["labels"] != -100
            valid_preds = preds[mask]
            valid_labels = inputs["labels"][mask]

            # Handle possible unknown labels
            for p, t in zip(valid_preds, valid_labels):
                p_id = p.item()
                t_id = t.item()

                try:
                    p_label = id2label[p_id]
                except KeyError:
                    p_label = "UNK"

                try:
                    t_label = id2label[t_id]
                except KeyError:
                    continue  # Skip invalid labels in ground truth

                if p_label == t_label:
                    class_counts[t_label]['tp'] += 1
                else:
                    if p_label in class_counts:
                        class_counts[p_label]['fp'] += 1
                    class_counts[t_label]['fn'] += 1

            total_correct += (valid_preds == valid_labels).sum().item()
            total_tokens += mask.sum().item()

    # Rest of the evaluation logic remains the same...

    # Calculate metrics
    accuracy = total_correct / total_tokens
    f1_scores = {}
    for label, counts in class_counts.items():
        precision = counts['tp'] / (counts['tp'] + counts['fp']) if (counts['tp'] + counts['fp']) > 0 else 0
        recall = counts['tp'] / (counts['tp'] + counts['fn']) if (counts['tp'] + counts['fn']) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores[label] = f1

    print(f"Accuracy: {accuracy:.4f}")
    print("F1 Scores:")
    for label, score in f1_scores.items():
        print(f"{label}: {score:.4f}")

    return accuracy, f1_scores

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
# train_data = load_conllu_data("/content/ug_udt-ud-train.conllu")
# train_dataset = POSDataset(train_data, tokenizer)
# Initialize label mappings with both datasets
label2id, id2label = create_label_mapping(
    "/content/ug_udt-ud-train.conllu",
    "/content/ug_udt-ud-test.conllu"
)

# Initialize datasets with explicit label mapping
train_dataset = POSDataset(
    load_conllu_data("/content/ug_udt-ud-train.conllu"),
    tokenizer
)

test_dataset = POSDataset(
    load_conllu_data("/content/ug_udt-ud-test.conllu"),
    tokenizer
)
# Create model
model = UniPELTFusionPlusPOS(num_labels=len(train_dataset.label_map)).cuda()

# Train and evaluate
train_pos_model(model, train_loader, num_epochs=10)


evaluate_pos(model, test_loader, train_dataset.id2label)

# Usage remains the same as original
test_sentence = "نەشپۈت بەش يىلدا، ئۆرۈك تۆت يىلدا مېۋە بېرىدۇ دېگەننى ئاڭلىمىغانمىدىڭ؟"
predictions = predict_pos(test_sentence, model, tokenizer, train_dataset.id2label)

# Print results
for word, tag in predictions:
    print(f"{word:20} -> {tag}")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert

Epoch 1 Loss: 2.0484
Epoch 2 Loss: 2.0020
Epoch 3 Loss: 1.9010
Epoch 4 Loss: 1.9064
Epoch 5 Loss: 1.8183
Epoch 6 Loss: 1.8001
Epoch 7 Loss: 1.8067
Epoch 8 Loss: 1.7655
Epoch 9 Loss: 1.7183
Epoch 10 Loss: 1.6385
Epoch 11 Loss: 1.6671
Epoch 12 Loss: 1.6087
Epoch 13 Loss: 1.6569
Epoch 14 Loss: 1.6354
Epoch 15 Loss: 1.5842
Accuracy: 0.0302
F1 Scores:
ADJ: 0.0000
ADV: 0.0000
AUX: 0.0000
INTJ: 0.0000
NOUN: 0.0292
NUM: 0.0000
PRON: 0.0204
PROPN: 0.0000
PUNCT: 0.0588
VERB: 0.1000
نەشپۈت               -> NOUN
بەش                  -> NOUN
يىلدا،               -> NOUN
ئۆرۈك                -> NOUN
تۆت                  -> NOUN
يىلدا                -> NOUN
مېۋە                 -> NOUN
بېرىدۇ               -> NOUN
دېگەننى              -> NOUN
ئاڭلىمىغانمىدىڭ؟     -> NOUN


# Unipelt plus dependecy parsing

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from peft import LoraConfig, PrefixTuningConfig, get_peft_model, TaskType

class UniPELTFusionPlus(nn.Module):
    def __init__(self, hidden_size, num_scripts=2):
        super().__init__()
        # Enhanced fusion with layer normalization
        self.lora_fusion = nn.Sequential(
            nn.Linear(hidden_size * num_scripts, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Linear(256, num_scripts),
            nn.Softmax(dim=-1)
        )
        self.prefix_fusion = nn.Sequential(
            nn.Linear(hidden_size * num_scripts, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Linear(256, num_scripts),
            nn.Softmax(dim=-1)
        )

    def forward(self, script_components):
        # Dynamic fusion with residual connection
        lora_s, lora_t = script_components['lora']
        prefix_s, prefix_t = script_components['prefix']

        # LoRA fusion
        lora_cat = torch.cat([lora_s, lora_t], dim=-1)
        lora_weights = self.lora_fusion(lora_cat)
        fused_lora = lora_weights[:,:,0:1]*lora_s + lora_weights[:,:,1:2]*lora_t

        # Prefix fusion
        prefix_cat = torch.cat([prefix_s, prefix_t], dim=-1)
        prefix_weights = self.prefix_fusion(prefix_cat)
        fused_prefix = prefix_weights[:,:,0:1]*prefix_s + prefix_weights[:,:,1:2]*prefix_t

        return fused_lora + fused_prefix + 0.5*(lora_s + lora_t + prefix_s + prefix_t)

class UniPELTDependencyParser(nn.Module):
    def __init__(self, num_dep_labels, max_seq_length=128):
        super().__init__()
        self.max_seq_length = max_seq_length

        # Shared base model
        self.base_model = AutoModel.from_pretrained("bert-base-multilingual-cased")

        # Freeze base model
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Script-specific adapters
        self.script_configs = nn.ModuleDict({
            script: nn.ModuleDict({
                'lora': get_peft_model(
                    AutoModel.from_pretrained("bert-base-multilingual-cased"),
                    LoraConfig(
                        r=8,
                        lora_alpha=32,
                        target_modules=["query", "value"],
                        bias="none",
                        task_type=TaskType.FEATURE_EXTRACTION
                    )
                ),
                'prefix': get_peft_model(
                    AutoModel.from_pretrained("bert-base-multilingual-cased"),
                    PrefixTuningConfig(
                        num_virtual_tokens=20,
                        task_type=TaskType.FEATURE_EXTRACTION,
                        prefix_projection=False
                    )
                )
            }) for script in ['s', 't']
        })

        # Fusion and prediction
        self.fusion = UniPELTFusionPlus(self.base_model.config.hidden_size)
        self.head_predictor = nn.Linear(self.base_model.config.hidden_size, max_seq_length)
        self.label_predictor = nn.Linear(self.base_model.config.hidden_size, num_dep_labels)
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask):
        script_outputs = {'lora': [], 'prefix': []}

        for script in ['s', 't']:
            # LoRA pathway
            lora_out = self.script_configs[script]['lora'](
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True
            ).hidden_states[-1]

            # Prefix pathway
            prefix_out = self.script_configs[script]['prefix'](
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True
            ).hidden_states[-1]

            script_outputs['lora'].append(lora_out)
            script_outputs['prefix'].append(prefix_out)

        # Fuse representations
        fused = self.fusion(script_outputs)
        fused = self.dropout(fused)

        # Predict dependencies
        head_logits = self.head_predictor(fused)
        label_logits = self.label_predictor(fused)

        return head_logits, label_logits

# Initialize model with proper parameter isolation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dep_label2id, dep_id2label = create_dep_label_mapping(conllu_pth)
model = UniPELTDependencyParser(
    num_dep_labels=len(dep_label2id),
    max_seq_length=128
).to(device)

# Enhanced training with gradient accumulation
def train_unipelt_fusion(model, dataloader, num_epochs=3, accum_steps=4):
    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=2e-5,
        weight_decay=0.01
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=num_epochs*len(dataloader)//accum_steps
    )

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()

        for i, batch in enumerate(dataloader):
            inputs = {k: v.to(device) for k, v in batch.items()}
            head_logits, label_logits = model(inputs["input_ids"], inputs["attention_mask"])

            # Calculate losses
            head_loss = F.cross_entropy(
                head_logits.view(-1, model.max_seq_length),
                inputs["dep_heads"].view(-1),
                ignore_index=-1
            )
            label_loss = F.cross_entropy(
                label_logits.view(-1, label_logits.size(-1)),
                inputs["dep_labels"].view(-1),
                ignore_index=-1
            )
            loss = (head_loss + label_loss) / accum_steps
            loss.backward()

            if (i+1) % accum_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

            total_loss += loss.item() * accum_steps

        print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader):.4f}")

# Enhanced evaluation with tree validation
def evaluate_model(model, dataloader):
    model.eval()
    metrics = {'uas': 0, 'las': 0, 'total': 0}

    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            head_logits, label_logits = model(inputs["input_ids"], inputs["attention_mask"])

            # Get predictions
            head_preds = torch.argmax(head_logits, dim=-1)
            label_preds = torch.argmax(label_logits, dim=-1)

            # Calculate valid tokens
            mask = (inputs["dep_labels"] != -1) & (inputs["dep_heads"] != -1)
            metrics['total'] += mask.sum().item()

            # Calculate matches
            correct_head = (head_preds[mask] == inputs["dep_heads"][mask])
            correct_label = (label_preds[mask] == inputs["dep_labels"][mask])
            metrics['uas'] += correct_head.sum().item()
            metrics['las'] += (correct_head & correct_label).sum().item()

    uas = metrics['uas'] / metrics['total']
    las = metrics['las'] / metrics['total']
    print(f"Unlabeled Attachment Score: {uas:.4f}")
    print(f"Labeled Attachment Score: {las:.4f}")
    return uas, las
# Initialize data loaders
train_loader = DataLoader(
    ConlluDataset(conllu_pth, tokenizer, dep_label2id),
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)
val_loader = DataLoader(
    ConlluDataset("/content/ug_udt-ud-dev.conllu", tokenizer, dep_label2id),
    batch_size=2,
    collate_fn=collate_fn
)

In [ ]:
# Train and evaluate
train_unipelt_fusion(model, train_loader, num_epochs=100)
evaluate_model(model, val_loader)

Epoch 1 Loss: 9.0337
Epoch 2 Loss: 8.8239
Epoch 3 Loss: 8.7601
Epoch 4 Loss: 8.8130
Epoch 5 Loss: 8.6873
Epoch 6 Loss: 8.6392
Epoch 7 Loss: 8.5404
Epoch 8 Loss: 8.4710
Epoch 9 Loss: 8.4333
Epoch 10 Loss: 8.4580
Epoch 11 Loss: 8.3892
Epoch 12 Loss: 8.2598
Epoch 13 Loss: 8.1667
Epoch 14 Loss: 8.1342
Epoch 15 Loss: 7.9716
Epoch 16 Loss: 8.0880
Epoch 17 Loss: 7.9655
Epoch 18 Loss: 8.0396
Epoch 19 Loss: 7.8549
Epoch 20 Loss: 7.8566
Epoch 21 Loss: 7.7258
Epoch 22 Loss: 7.7988
Epoch 23 Loss: 7.6627
Epoch 24 Loss: 7.6241
Epoch 25 Loss: 7.5606
Epoch 26 Loss: 7.4905
Epoch 27 Loss: 7.4605
Epoch 28 Loss: 7.4145
Epoch 29 Loss: 7.4700
Epoch 30 Loss: 7.3573
Epoch 31 Loss: 7.2174
Epoch 32 Loss: 7.2355
Epoch 33 Loss: 7.2675
Epoch 34 Loss: 7.1396
Epoch 35 Loss: 7.1354
Epoch 36 Loss: 7.0493
Epoch 37 Loss: 7.0380
Epoch 38 Loss: 7.0553
Epoch 39 Loss: 6.9847
Epoch 40 Loss: 6.9403
Epoch 41 Loss: 6.8762
Epoch 42 Loss: 6.8361
Epoch 43 Loss: 6.7998
Epoch 44 Loss: 6.9041
Epoch 45 Loss: 6.8001
Epoch 46 Loss: 6.70

(0.06796116504854369, 0.009708737864077669)

In [ ]:
def parse_sentence_dep_fusion(model, tokenizer, sentence, dep_id2label, max_length=128):
    # Tokenize input
    words = sentence.split()
    encoding = tokenizer(
        words,
        is_split_into_words=True,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        return_offsets_mapping=True
    )

    # Get predictions
    with torch.no_grad():
        head_logits, label_logits = model(
            encoding["input_ids"].to(device),
            encoding["attention_mask"].to(device)
        )

    # Convert to predictions
    head_preds = torch.argmax(head_logits, dim=-1).squeeze().cpu().numpy()
    label_preds = torch.argmax(label_logits, dim=-1).squeeze().cpu().numpy()

    # Align with original words
    word_ids = encoding.word_ids()
    dependencies = []
    current_word = None

    for idx, wid in enumerate(word_ids):
        if wid is None or wid == current_word:
            continue  # Skip special tokens and subwords

        current_word = wid
        try:
            dep_entry = (
                words[wid],  # Token
                int(head_preds[idx]),  # Predicted head
                dep_id2label[label_preds[idx]]  # Dependency label
            )
            dependencies.append(dep_entry)
        except IndexError:
            continue

    # Validate tree structure
    root = [i for i, (_, head, _) in enumerate(dependencies) if head == 0]
    if not root:
        dependencies[0] = (dependencies[0][0], 0, "root")

    return dependencies

# Example usage
test_sentence = "نەشپۈت بەش يىلدا، ئۆرۈك تۆت يىلدا مېۋە بېرىدۇ دېگەننى ئاڭلىمىغانمىدىڭ؟"
dependencies = parse_sentence_dep_fusion(model, tokenizer, test_sentence, dep_id2label)

print("Token\tHead\tDependency")
for i, (token, head, label) in enumerate(dependencies):
    print(f"{i+1}\t{token}\t{head}\t{label}")

Token	Head	Dependency
1	نەشپۈت	15	vocative
2	بەش	15	vocative
3	يىلدا،	12	vocative
4	ئۆرۈك	15	vocative
5	تۆت	0	vocative
6	يىلدا	58	obl
7	مېۋە	42	vocative
8	بېرىدۇ	15	vocative
9	دېگەننى	15	vocative
10	ئاڭلىمىغانمىدىڭ؟	58	orphan
